In [ ]:
from datasets import SOCOFingGender, SOCOFingFingers, SOCOFingSubjects, SOCOFingHeands, QFing
import FPMLmodule.backbones as backbones
import FPMLmodule.classifiers as classifiers
from FPMLmodule.fpml import FPML 
import FPMLmodule.utils as utils
import numpy as np
from matplotlib import pyplot as plt
# from keras import layers, Sequential, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

In [ ]:
# Global Config
seed=9
imgDim = (120, 120, 3)
imgHeight, imgWidth, imgChannels = imgDim
batchSize = 32

# Dataset configuration
dsConfig = {
    'batchSize': batchSize, 
    'parallelTune': tf.data.AUTOTUNE, 
    'split': [0.7, 0.15, 0.15], 
    'inputDim': imgDim, 
    'seed': seed, 
    'shuffle': True
    }

weightsRN50 = "./weights/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsMNV2 = "./weights/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5"
weightsENB2 = "./weights/efficientnetb2_notop.h5"
weightsINCEPTIONV3 = "./weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
weightsXCEPTION = "./weights/xception_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Training interval
epochsForSearch = 10
epochsForBest = 100

In [ ]:
SOCOGender = SOCOFingGender(**dict(dsConfig, sampling=SOCOFingGender.UNDER_SAMPLING))
SOCOSubjects = SOCOFingSubjects(**dsConfig)
SOCOFingers = SOCOFingFingers(**dsConfig)
QualityFing = QFing(**dsConfig)
SOCOHeands = SOCOFingHeands(**dsConfig)

genderDS = SOCOGender.create()
subjectDS = SOCOSubjects.create()
fingersDS = SOCOFingers.create()
heandsDS = SOCOHeands.create()
qfingDS = QualityFing.create()

In [ ]:
# utils.displayDsSamples(subjectDS[0], 'Heands - Train', (5, 5), classNames=None)

In [ ]:
train_ds, test_ds, val_ds = genderDS
nbClasses = len(SOCOFingers.classNames)
activation = "softmax"

hypers = {
    "optimizer": Adam,
    "learningRate": 0.001,
    "loss": 'binary_crossentropy',
    "metrics": 'accuracy'
}
trainingModeStudy = {
    "mobilenetV2-pretrained-untrainable" : {
        "architecture": {
            "backbone": backbones.ResNet50(imgDim, weights=weightsMNV2, trainable=False),
            "classfier": classifiers.DefaultClassifier(nbClasses, activation),
            "inputLayer": None, 
            "inputDim": imgDim
        },
        "hyperparameters":hypers
    }
}
# studies
# rn50 = ResNet50(img_dim, weights=weights, trainable=False)
# rnc = ResNetClassifier(2, "softmax")
# fpml = FPML(rn50, rnc, "", img_dim)
# model = fpml.create(Adam, learning_rate, 'binary_crossentropy', 'accuracy')
# model_history = model.fit(train_ds, validation_data=val_ds, epochs=10)
# test_history = model.evaluate(test_ds)

In [ ]:
def researchStudies(train_set, val_set, studies, epochs=10, verbose=2):
    histories = {}
    for study, config in studies.items():
        fpml = FPML(**config["architecture"])
        model = fpml.create(**config["hyperparameters"])
        model_history = model.fit(train_set, validation_data=val_set, epochs=epochs, verbose=verbose)
        histories[study] = model_history
    return histories

In [ ]:
# utils.displayTrainProgress(model_history)
# test_acc = utils.displayConfusion(test_ds, model)

In [ ]:
# from tensorflow.keras import models
# from tensorflow.keras import layers
# from tensorflow import preprocessing


# trainAug = models.Sequential([
# 	preprocessing.Rescaling(scale=1.0 / 255),
# 	preprocessing.RandomFlip("horizontal_and_vertical"),
# 	preprocessing.RandomZoom(
# 		height_factor=(-0.05, -0.15),
# 		width_factor=(-0.05, -0.15)),
# 	preprocessing.RandomRotation(0.3)
# ])


# augmanted = data_augmentation(genderDS[0])
